# 2.2: Project Planning and Sourcing Data with an API

## Imports & Settings 

In [1]:
import pandas as pd
from pathlib import Path
import requests
from datetime import datetime

## Load Citibike Data

In [2]:
data_folder = Path(r"C:\Users\cneva\Projects\CitiBike Analysis\Raw Data\2022-citibike-tripdata")

citibike_data = pd.concat([ # concatenate the lists of dataframes
    pd.read_csv(file_path, parse_dates = ["started_at", "ended_at"], low_memory = False) # read the csv files using list comprehension
      .assign(source = file_path.stem)  # add column for the source file name
    for file_path in data_folder.rglob("*.csv")
], ignore_index = True)

In [3]:
# clean data
citibike_df = (
    citibike_data
    .assign(
        start_station_id = pd.to_numeric(citibike_data["start_station_id"], errors = "coerce"), # convert columns to numeric, invalid cells convert to NaN
        end_station_id = pd.to_numeric(citibike_data["end_station_id"], errors = "coerce"),
        date = citibike_data["started_at"].dt.normalize(),
    )
)

In [5]:
citibike_df

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,source,date
0,BFD29218AB271154,electric_bike,2022-01-21 13:13:43.392,2022-01-21 13:22:31.463,West End Ave & W 107 St,7650.05,Mt Morris Park W & W 120 St,7685.14,40.802117,-73.968181,40.804038,-73.945925,member,202201-citibike-tripdata_1,2022-01-21
1,7C953F2FD7BE1302,classic_bike,2022-01-10 11:30:54.162,2022-01-10 11:41:43.422,4 Ave & 3 St,4028.04,Boerum Pl\t& Pacific St,4488.09,40.673746,-73.985649,40.688489,-73.991160,member,202201-citibike-tripdata_1,2022-01-10
2,95893ABD40CED4B8,electric_bike,2022-01-26 10:52:43.096,2022-01-26 11:06:35.227,1 Ave & E 62 St,6753.08,5 Ave & E 29 St,6248.06,40.761227,-73.960940,40.745168,-73.986831,member,202201-citibike-tripdata_1,2022-01-26
3,F853B50772137378,classic_bike,2022-01-03 08:35:48.247,2022-01-03 09:10:50.475,2 Ave & E 96 St,7338.02,5 Ave & E 29 St,6248.06,40.783964,-73.947167,40.745168,-73.986831,member,202201-citibike-tripdata_1,2022-01-03
4,7590ADF834797B4B,classic_bike,2022-01-22 14:14:23.043,2022-01-22 14:34:57.474,6 Ave & W 34 St,6364.10,5 Ave & E 29 St,6248.06,40.749640,-73.988050,40.745168,-73.986831,member,202201-citibike-tripdata_1,2022-01-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29838801,1F223EDAFF420AE3,electric_bike,2022-12-01 20:26:45.847,2022-12-01 20:30:46.012,Avenue D & E 3 St,5436.09,Stanton St & Chrystie St,5523.02,40.720701,-73.977939,40.722293,-73.991475,member,202212-citibike-tripdata_2,2022-12-01
29838802,CFA5C560ACB73B8E,classic_bike,2022-12-26 13:46:34.237,2022-12-26 13:52:43.900,43 Ave & 47 St,6209.05,39 Ave & 45 St,6401.03,40.744806,-73.917290,40.749478,-73.918265,member,202212-citibike-tripdata_2,2022-12-26
29838803,11C8C5E0DB947B07,classic_bike,2022-12-01 05:56:14.903,2022-12-01 06:06:10.357,Avenue D & E 3 St,5436.09,Bleecker St & Crosby St,5679.08,40.720828,-73.977932,40.726156,-73.995102,member,202212-citibike-tripdata_2,2022-12-01
29838804,5B9B083C534A5964,classic_bike,2022-12-02 11:54:15.871,2022-12-02 12:01:00.747,Montague St & Clinton St,4677.06,Sands St & Jay St,4821.03,40.694271,-73.992327,40.700119,-73.986200,member,202212-citibike-tripdata_2,2022-12-02


In [6]:
citibike_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29838806 entries, 0 to 29838805
Data columns (total 15 columns):
 #   Column              Dtype         
---  ------              -----         
 0   ride_id             object        
 1   rideable_type       object        
 2   started_at          datetime64[ns]
 3   ended_at            datetime64[ns]
 4   start_station_name  object        
 5   start_station_id    float64       
 6   end_station_name    object        
 7   end_station_id      float64       
 8   start_lat           float64       
 9   start_lng           float64       
 10  end_lat             float64       
 11  end_lng             float64       
 12  member_casual       object        
 13  source              object        
 14  date                datetime64[ns]
dtypes: datetime64[ns](3), float64(6), object(6)
memory usage: 3.3+ GB


## Load NOAA temperature data

In [8]:
# read in NOAA API Token
with open(r"noaa_token\noaa_token.txt", "r") as file:
    noaa_token = file.read().strip()

### Access API weather data

In [10]:
# calculate min and max dates in the Citibike dataset to pull the correct weather dates
citibike_df.started_at.min()

Timestamp('2021-01-30 17:30:45.544000')

In [11]:
citibike_df.ended_at.max()

Timestamp('2022-12-31 23:59:55.708000')

In [12]:
base_url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/data"

params = {
    "datasetid": "GHCND",
    "datatypeid": "TAVG",
    "limit": 1000,
    "stationid": "GHCND:USW00014732", #LaGuardia ID GHCND:USW00014732 GHCND:USW00094846
    "startdate": "2022-01-01",
    "enddate": "2022-12-31"
}

headers = {"token": noaa_token}

response = requests.get(base_url, headers = headers, params = params)

# parse the response
if response.status_code == 200:
    data = response.json()
else:
    print("Error:", response.status_code, response.text)

In [21]:
data

{'metadata': {'resultset': {'offset': 1, 'count': 365, 'limit': 1000}},
 'results': [{'date': '2022-01-01T00:00:00',
   'datatype': 'TAVG',
   'station': 'GHCND:USW00014732',
   'attributes': 'H,,S,',
   'value': 116},
  {'date': '2022-01-02T00:00:00',
   'datatype': 'TAVG',
   'station': 'GHCND:USW00014732',
   'attributes': 'H,,S,',
   'value': 114},
  {'date': '2022-01-03T00:00:00',
   'datatype': 'TAVG',
   'station': 'GHCND:USW00014732',
   'attributes': 'H,,S,',
   'value': 14},
  {'date': '2022-01-04T00:00:00',
   'datatype': 'TAVG',
   'station': 'GHCND:USW00014732',
   'attributes': 'H,,S,',
   'value': -27},
  {'date': '2022-01-05T00:00:00',
   'datatype': 'TAVG',
   'station': 'GHCND:USW00014732',
   'attributes': 'H,,S,',
   'value': 32},
  {'date': '2022-01-06T00:00:00',
   'datatype': 'TAVG',
   'station': 'GHCND:USW00014732',
   'attributes': 'H,,S,',
   'value': 49},
  {'date': '2022-01-07T00:00:00',
   'datatype': 'TAVG',
   'station': 'GHCND:USW00014732',
   'attribut

### Clean & Prep Data

In [24]:
temps_df = pd.DataFrame(
    item for item in json.loads(response.text)["results"]
    if item["datatype"] == "TAVG"
)

temps_df = (
    temps_df
    .assign(
        date = temps_df["date"].apply(lambda d: datetime.strptime(d, "%Y-%m-%dT%H:%M:%S")),
        avgTempCelsius = temps_df["value"] / 10.0
    )
    .loc[:, ["date", "avgTempCelsius"]]
)


In [26]:
temps_df

,date,avgTempCelsius
0,2022-01-01,11.6
1,2022-01-02,11.4
2,2022-01-03,1.4
3,2022-01-04,-2.7
4,2022-01-05,3.2
...,...,...
360,2022-12-27,-0.7
361,2022-12-28,3.4
362,2022-12-29,6.4
363,2022-12-30,9.3


In [28]:
citibike_df.dtypes

ride_id                       object
rideable_type                 object
started_at            datetime64[ns]
ended_at              datetime64[ns]
start_station_name            object
start_station_id             float64
end_station_name              object
end_station_id               float64
start_lat                    float64
start_lng                    float64
end_lat                      float64
end_lng                      float64
member_casual                 object
source                        object
date                  datetime64[ns]
dtype: object

In [30]:
temps_df.dtypes

date              datetime64[ns]
avgTempCelsius           float64
dtype: object

## Combine the Datasets

In [33]:
combined_df = citibike_df.merge(temps_df, how = "left", left_on = "date", right_on = "date", indicator = True)

combined_df["_merge"].value_counts(dropna = False)

both          29838166
left_only          640
right_only           0
Name: _merge, dtype: int64

In [34]:
combined_df[combined_df["_merge"] == "left_only"]

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,source,date,avgTempCelsius,_merge
4967,9D2DEF9B6D4FE1F2,electric_bike,2021-12-31 23:56:04.967,2022-01-01 00:10:03.697,3 St & Prospect Park West,3865.05,Atlantic Ave & Furman St,4614.04,40.668132,-73.973638,40.691652,-73.999979,member,202201-citibike-tripdata_1,2021-12-31,NaN,left_only
8335,07F3FEAE54546CCC,classic_bike,2021-12-31 23:25:37.862,2022-01-01 00:40:26.455,Hudson St & W 13 St,6115.06,W 15 St & 6 Ave,5989.02,40.740057,-74.005274,40.738046,-73.996430,casual,202201-citibike-tripdata_1,2021-12-31,NaN,left_only
9102,F7054D918A815DED,electric_bike,2021-12-31 23:38:49.704,2022-01-01 00:29:24.363,E 85 St & 3 Ave,7212.05,E 81 St & 2 Ave,7121.05,40.778012,-73.954071,40.774779,-73.954275,member,202201-citibike-tripdata_1,2021-12-31,NaN,left_only
9178,302DA77D1C2677A4,electric_bike,2021-12-31 23:41:16.311,2022-01-01 00:03:43.417,Perry St & Greenwich Ave,5955.12,E 48 St & 3 Ave,6541.03,40.735918,-74.000939,40.754601,-73.971879,casual,202201-citibike-tripdata_1,2021-12-31,NaN,left_only
18361,7A6CD4B02003DA01,classic_bike,2021-12-31 23:45:39.596,2022-01-01 00:09:44.400,W 84 St & Columbus Ave,7382.04,Lexington Ave & E 120 St,7652.04,40.785000,-73.972834,40.801307,-73.939817,casual,202201-citibike-tripdata_1,2021-12-31,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21775695,763D82F8CFA1D9D1,classic_bike,2021-03-31 15:35:20.070,2022-09-25 14:33:21.607,Willoughby St & Fleet St,4628.05,DeKalb Ave & Franklin Ave,4528.01,40.691966,-73.981302,40.690648,-73.957462,casual,202209-citibike-tripdata_3,2021-03-31,NaN,left_only
26999657,799622522BCC91B3,classic_bike,2021-09-05 17:52:23.693,2022-11-16 06:18:23.556,Anderson Ave & W 164 St,8096.03,Pier 40 Dock Station,NaN,40.832057,-73.927421,40.728660,-74.011980,casual,202211-citibike-tripdata_2,2021-09-05,NaN,left_only
27158219,55DE96B9AF7BFDF6,classic_bike,2021-07-19 14:41:06.067,2022-11-12 13:25:43.737,Fulton St & Pearl St,5024.09,E 102 St & 1 Ave,7407.13,40.707722,-74.004386,40.786995,-73.941648,casual,202211-citibike-tripdata_2,2021-07-19,NaN,left_only
28515550,87D33069747BE58B,classic_bike,2021-10-27 03:55:34.375,2022-12-13 04:25:16.144,Greene Ave & Throop Ave,4510.04,Ashland Pl & Dekalb Ave,4513.09,40.689493,-73.942061,40.690065,-73.978776,casual,202212-citibike-tripdata_1,2021-10-27,NaN,left_only


In [35]:
df = combined_df.drop(columns = ["date", "_merge"])

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29838806 entries, 0 to 29838805
Data columns (total 15 columns):
 #   Column              Dtype         
---  ------              -----         
 0   ride_id             object        
 1   rideable_type       object        
 2   started_at          datetime64[ns]
 3   ended_at            datetime64[ns]
 4   start_station_name  object        
 5   start_station_id    float64       
 6   end_station_name    object        
 7   end_station_id      float64       
 8   start_lat           float64       
 9   start_lng           float64       
 10  end_lat             float64       
 11  end_lng             float64       
 12  member_casual       object        
 13  source              object        
 14  avgTempCelsius      float64       
dtypes: datetime64[ns](2), float64(7), object(6)
memory usage: 3.6+ GB


In [25]:
# df.to_csv("2022_Citibike_Dataset.csv", index = False)

In [38]:
df.to_parquet("2022_Citibike_Dataset.parquet", engine = "pyarrow")

## 2.7

In [28]:
station_df = \
(df
 .query("started_at >= '2022-01-01' and started_at < '2023-01-01'")
 .assign(
     duration_min = lambda df: (df["ended_at"] - df["started_at"]).dt.total_seconds() / 60,
     round_trip = df["start_station_name"] == df["end_station_name"])
 .query("duration_min <= 24 * 60")
 .melt(
     id_vars = "round_trip",
     value_vars = ["start_station_name", "end_station_name"],
     var_name = "type",
     value_name = "station"
 )
 .assign(
     start_count = lambda d: (d["type"].eq("start_station_name") & ~d["round_trip"]).astype(int),
     end_count = lambda d: d["type"].eq("end_station_name").astype(int),
     round_trip_count = lambda d: (d["type"].eq("start_station_name") & d["round_trip"]).astype(int),
 )
 .groupby("station", as_index = False)[["start_count", "end_count", "round_trip_count"]].sum()
)

station_df

,station,start_count,end_count,round_trip_count
0,1 Ave & E 110 St,20709,21620,791
1,1 Ave & E 16 St,66048,66390,1222
2,1 Ave & E 18 St,69233,70485,1547
3,1 Ave & E 30 St,44810,45992,564
4,1 Ave & E 39 St,51180,52693,1129
...,...,...,...,...
1836,Wyckoff St & 3 Ave,243,250,7
1837,Wyckoff St & Nevins St,11187,11609,327
1838,Wythe Ave & Metropolitan Ave,64738,67120,1643
1839,Yankee Ferry Terminal,7093,12855,5752


In [59]:
summary_df = (
    df
    .query("started_at >= '2022-01-01' and started_at < '2023-01-01'")
    .assign(duration_min = lambda df: (df["ended_at"] - df["started_at"]).dt.total_seconds() / 60)
    .query("duration_min <= 24 * 60")
    .loc[:, ["member_casual", "duration_min", "ride_id", "rideable_type"]]
    .pipe(lambda df: pd.DataFrame({
        "rental_count": [df["ride_id"].count()],
        "average_duration": [df["duration_min"].mean()],
        "member_perc": [(df["member_casual"] == "member").mean()],
        "bike_type_perc": [(df["rideable_type"] == "electric_bike").mean()],
    }))
    .assign(year = 2022)
)

summary_df

,rental_count,average_duration,member_perc,bike_type_perc,year
0,29798394,15.485299,0.779798,0.393368,2022


In [32]:
station_df.to_csv(r"Data/2022_station_df.csv", index = False)

In [62]:
summary_df.to_csv(r"Data/2022_summary_df.csv", index = False)